In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import time
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Custom Dataset class for PyTorch integration
class CoffeeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# PyTorch wrapper for sklearn RandomForest model
class RandomForestWrapper(nn.Module):
    def __init__(self, rf_model):
        super(RandomForestWrapper, self).__init__()
        self.rf_model = rf_model
        
    def forward(self, x):
        # Convert tensor to numpy for sklearn
        x_np = x.detach().cpu().numpy()
        
        # Get predictions from RF model
        pred_np = self.rf_model.predict(x_np)
        
        # Convert back to tensor
        pred_tensor = torch.tensor(pred_np, dtype=torch.float32).to(x.device)
        
        return pred_tensor

# Function to train a random forest model
def train_rf_model(X_train, y_train, X_test, y_test, multi_output=False, 
                  n_estimators=100, max_depth=None, n_jobs=-1):
    start_time = time.time()
    
    # Create and train model
    if multi_output:
        base_model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=n_jobs
        )
        model = MultiOutputRegressor(base_model)
    else:
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=n_jobs
        )
    
    # Reshape y_train for single output if needed
    if not multi_output and len(y_train.shape) > 1 and y_train.shape[1] == 1:
        y_train = y_train.ravel()
        y_test = y_test.ravel()
    
    # Train model
    model.fit(X_train, y_train)
    
    # Calculate R^2 score
    y_pred = model.predict(X_test)
    
    if multi_output:
        r2_scores = []
        for i in range(y_test.shape[1]):
            r2 = r2_score(y_test[:, i], y_pred[:, i])
            r2_scores.append(r2)
        mean_r2 = np.mean(r2_scores)
    else:
        if len(y_test.shape) > 1 and y_test.shape[1] == 1:
            y_test = y_test.ravel()
        r2_scores = [r2_score(y_test, y_pred)]
        mean_r2 = r2_scores[0]
    
    train_time = time.time() - start_time
    
    return model, r2_scores, mean_r2, train_time

# Function to optimize hyperparameters and train best model
def optimize_rf_model(X_train, y_train, X_test, y_test, multi_output=False, 
                    n_jobs=-1, target_name=""):
    print(f"Optimizing model for {target_name}...")
    
    best_r2 = -float('inf')
    best_params = None
    best_model = None
    best_r2_scores = None
    
    # Grid of parameters to try
    n_estimators_list = [50, 100, 200, 300]
    max_depth_list = [None, 10, 20, 30, 40, 50]
    min_samples_split_list = [2, 5, 10]
    
    # Try combinations
    for n_estimators in n_estimators_list:
        for max_depth in max_depth_list:
            for min_samples_split in min_samples_split_list:
                # Only try a subset of combinations to save time
                if n_estimators > 100 and max_depth is not None and max_depth > 30:
                    continue
                
                if multi_output:
                    base_model = RandomForestRegressor(
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        min_samples_split=min_samples_split,
                        random_state=42,
                        n_jobs=n_jobs
                    )
                    model = MultiOutputRegressor(base_model)
                else:
                    model = RandomForestRegressor(
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        min_samples_split=min_samples_split,
                        random_state=42,
                        n_jobs=n_jobs
                    )
                
                # Reshape y_train for single output if needed
                y_train_fit = y_train
                y_test_eval = y_test
                if not multi_output and len(y_train.shape) > 1 and y_train.shape[1] == 1:
                    y_train_fit = y_train.ravel()
                    y_test_eval = y_test.ravel()
                
                # Fit model
                model.fit(X_train, y_train_fit)
                
                # Evaluate model
                y_pred = model.predict(X_test)
                
                if multi_output:
                    r2_scores = []
                    for i in range(y_test.shape[1]):
                        r2 = r2_score(y_test[:, i], y_pred[:, i])
                        r2_scores.append(r2)
                    mean_r2 = np.mean(r2_scores)
                else:
                    if len(y_test_eval.shape) > 1 and y_test_eval.shape[1] == 1:
                        y_test_eval = y_test_eval.ravel()
                    r2_scores = [r2_score(y_test_eval, y_pred)]
                    mean_r2 = r2_scores[0]
                
                # Save best model
                if mean_r2 > best_r2:
                    best_r2 = mean_r2
                    best_params = {
                        'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split
                    }
                    best_model = model
                    best_r2_scores = r2_scores
                    
                    # Print progress
                    print(f"Found better model: R² = {mean_r2:.4f}, Params: {best_params}")
                    
                    # If we've achieved a really good score, we can stop early
                    if best_r2 > 0.85:
                        print("Found excellent model, stopping early.")
                        return best_model, best_r2_scores, best_r2, best_params
    
    return best_model, best_r2_scores, best_r2, best_params

# Function to plot feature importance
def plot_feature_importance(model, feature_names, title, multi_output=False):
    plt.figure(figsize=(12, 8))
    
    # Get feature importances
    if multi_output:
        # For multi-output, average across estimators
        importances = np.zeros(len(feature_names))
        for est in model.estimators_:
            importances += est.feature_importances_ / len(model.estimators_)
    else:
        importances = model.feature_importances_
    
    # Sort by importance
    indices = np.argsort(importances)[::-1]
    
    # Plot top 15 features
    n_features = min(15, len(feature_names))
    plt.title(f'Feature Importance: {title}')
    plt.barh(range(n_features), importances[indices[:n_features]])
    plt.yticks(range(n_features), [feature_names[i] for i in indices[:n_features]])
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_").lower()}_feature_importance.png')
    plt.close()
    
    # Return as dictionary
    return {feature_names[i]: importances[i] for i in indices}

# Function to make predictions
def make_predictions(model, X, multi_output=False):
    return model.predict(X)

# Main execution
print("Loading coffee dataset...")
df = pd.read_csv("final_coffee.csv")
print(f"Dataset shape: {df.shape}")

# Check for missing values
print(f"Checking for NaN values: {df.isna().sum().sum()}")

# Handle missing values if any
if df.isna().sum().sum() > 0:
    print("Filling missing values...")
    df = df.fillna(df.mean())

# Define features and targets
env_features = ["Annual precipitation", "Temperature Change on Land", "Altitude"]
processing_features = [col for col in df.columns if col.startswith("Processing Method_")]
origin_features = [col for col in df.columns if col.startswith("Country of Origin_")]
extra_cols = ["Category One Defects", "Quakers", "Category Two Defects"]

quality_input_cols = env_features + extra_cols + processing_features + origin_features
moisture_input_cols = env_features + processing_features + origin_features
yield_input_cols = env_features + processing_features + origin_features

quality_target_cols = ["Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance", 
                      "Uniformity", "Clean Cup", "Sweetness", "Cupper Points"]
moisture_target_col = "Moisture"
yield_target_col = "Total Yield"

# Prepare data

# Moisture Model
print("\n===== Training Moisture Model =====")
X_moisture = df[moisture_input_cols].values
y_moisture = df[[moisture_target_col]].values

# Split data
X_moisture_train, X_moisture_test, y_moisture_train, y_moisture_test = train_test_split(
    X_moisture, y_moisture, test_size=0.2, random_state=42
)

# Scale features
moisture_scaler = StandardScaler()
X_moisture_train_scaled = moisture_scaler.fit_transform(X_moisture_train)
X_moisture_test_scaled = moisture_scaler.transform(X_moisture_test)

# Train basic model first
moisture_model, moisture_r2_scores, moisture_r2, moisture_time = train_rf_model(
    X_moisture_train_scaled, y_moisture_train, X_moisture_test_scaled, y_moisture_test,
    multi_output=False, n_estimators=100
)

print(f"Moisture Model R²: {moisture_r2:.4f} (trained in {moisture_time:.2f} seconds)")

# Yield Model
print("\n===== Training Yield Model =====")
X_yield = df[yield_input_cols].values
y_yield = df[[yield_target_col]].values

# Split data
X_yield_train, X_yield_test, y_yield_train, y_yield_test = train_test_split(
    X_yield, y_yield, test_size=0.2, random_state=42
)

# Scale features
yield_scaler = StandardScaler()
X_yield_train_scaled = yield_scaler.fit_transform(X_yield_train)
X_yield_test_scaled = yield_scaler.transform(X_yield_test)

# Train basic model first
yield_model, yield_r2_scores, yield_r2, yield_time = train_rf_model(
    X_yield_train_scaled, y_yield_train, X_yield_test_scaled, y_yield_test,
    multi_output=False, n_estimators=100
)

print(f"Yield Model R²: {yield_r2:.4f} (trained in {yield_time:.2f} seconds)")

# Quality Model
print("\n===== Training Quality Model =====")
X_quality = df[quality_input_cols].values
y_quality = df[quality_target_cols].values

# Split data
X_quality_train, X_quality_test, y_quality_train, y_quality_test = train_test_split(
    X_quality, y_quality, test_size=0.2, random_state=42
)

# Scale features
quality_scaler = StandardScaler()
X_quality_train_scaled = quality_scaler.fit_transform(X_quality_train)
X_quality_test_scaled = quality_scaler.transform(X_quality_test)

# Train basic model first
quality_model, quality_r2_scores, quality_r2, quality_time = train_rf_model(
    X_quality_train_scaled, y_quality_train, X_quality_test_scaled, y_quality_test,
    multi_output=True, n_estimators=100
)

print(f"Quality Model Mean R²: {quality_r2:.4f} (trained in {quality_time:.2f} seconds)")
print("Individual R² scores:")
for i, col in enumerate(quality_target_cols):
    print(f"  {col}: {quality_r2_scores[i]:.4f}")

# Define the R² threshold for model optimization
r2_threshold = 0.7

# Optimize models if needed
if moisture_r2 < r2_threshold:
    print("\n===== Optimizing Moisture Model =====")
    moisture_model, moisture_r2_scores, moisture_r2, moisture_params = optimize_rf_model(
        X_moisture_train_scaled, y_moisture_train, X_moisture_test_scaled, y_moisture_test,
        multi_output=False, target_name="Moisture"
    )
    print(f"Optimized Moisture Model R²: {moisture_r2:.4f}")
    print(f"Best parameters: {moisture_params}")

if yield_r2 < r2_threshold:
    print("\n===== Optimizing Yield Model =====")
    yield_model, yield_r2_scores, yield_r2, yield_params = optimize_rf_model(
        X_yield_train_scaled, y_yield_train, X_yield_test_scaled, y_yield_test,
        multi_output=False, target_name="Yield"
    )
    print(f"Optimized Yield Model R²: {yield_r2:.4f}")
    print(f"Best parameters: {yield_params}")

if quality_r2 < r2_threshold:
    print("\n===== Optimizing Quality Model =====")
    quality_model, quality_r2_scores, quality_r2, quality_params = optimize_rf_model(
        X_quality_train_scaled, y_quality_train, X_quality_test_scaled, y_quality_test,
        multi_output=True, target_name="Quality"
    )
    print(f"Optimized Quality Model Mean R²: {quality_r2:.4f}")
    print(f"Best parameters: {quality_params}")
    print("Individual R² scores:")
    for i, col in enumerate(quality_target_cols):
        print(f"  {col}: {quality_r2_scores[i]:.4f}")

# Feature importance analysis
print("\n===== Feature Importance Analysis =====")

# Moisture model feature importance
moisture_importance = plot_feature_importance(
    moisture_model, moisture_input_cols, "Moisture Model"
)
print("\nTop 5 features for Moisture Model:")
for i, (feature, importance) in enumerate(sorted(moisture_importance.items(), 
                                             key=lambda x: x[1], reverse=True)[:5]):
    print(f"  {i+1}. {feature}: {importance:.4f}")

# Yield model feature importance
yield_importance = plot_feature_importance(
    yield_model, yield_input_cols, "Yield Model"
)
print("\nTop 5 features for Yield Model:")
for i, (feature, importance) in enumerate(sorted(yield_importance.items(), 
                                            key=lambda x: x[1], reverse=True)[:5]):
    print(f"  {i+1}. {feature}: {importance:.4f}")

# Quality model feature importance
if hasattr(quality_model, 'estimators_'):
    # For MultiOutputRegressor, use first estimator for feature importance
    quality_base_model = quality_model.estimators_[0]
    quality_importance = plot_feature_importance(
        quality_base_model, quality_input_cols, "Quality Model"
    )
    print("\nTop 5 features for Quality Model:")
    for i, (feature, importance) in enumerate(sorted(quality_importance.items(), 
                                              key=lambda x: x[1], reverse=True)[:5]):
        print(f"  {i+1}. {feature}: {importance:.4f}")

# Wrap the sklearn models with PyTorch for compatibility
moisture_pt_model = RandomForestWrapper(moisture_model)
yield_pt_model = RandomForestWrapper(yield_model)
quality_pt_model = RandomForestWrapper(quality_model)

# Sample predictions
print("\n===== Sample Predictions =====")
sample_data = df.sample(5, random_state=42)

# Prepare data for predictions
sample_moisture_X = moisture_scaler.transform(sample_data[moisture_input_cols].values)
sample_yield_X = yield_scaler.transform(sample_data[yield_input_cols].values)
sample_quality_X = quality_scaler.transform(sample_data[quality_input_cols].values)

# Make predictions
moisture_preds = make_predictions(moisture_model, sample_moisture_X)
yield_preds = make_predictions(yield_model, sample_yield_X)
quality_preds = make_predictions(quality_model, sample_quality_X)

# Display predictions
print("\nSample Predictions:")
for i in range(len(sample_data)):
    print(f"\nSample {i+1}:")
    print(f"  Altitude: {sample_data['Altitude'].iloc[i]:.2f}")
    
    # Get actual and predicted values (handling scalar vs array output)
    moisture_actual = sample_data[moisture_target_col].iloc[i]
    moisture_pred = moisture_preds[i] if moisture_preds.ndim > 1 else moisture_preds[i]
    
    yield_actual = sample_data[yield_target_col].iloc[i]
    yield_pred = yield_preds[i] if yield_preds.ndim > 1 else yield_preds[i]
    
    print(f"  Moisture - Actual: {moisture_actual:.2f}, Predicted: {moisture_pred:.2f}")
    print(f"  Yield - Actual: {yield_actual:.2f}, Predicted: {yield_pred:.2f}")
    
    print("  Quality Metrics:")
    for j, col in enumerate(quality_target_cols):
        actual = sample_data[col].iloc[i]
        predicted = quality_preds[i, j]
        print(f"    {col}: Actual: {actual:.2f}, Predicted: {predicted:.2f}")

# # Save models
# print("\n===== Saving Models =====")
# joblib.dump(moisture_model, 'moisture_rf_model.pkl')
# joblib.dump(yield_model, 'yield_rf_model.pkl')
# joblib.dump(quality_model, 'quality_rf_model.pkl')

# joblib.dump(moisture_scaler, 'moisture_scaler.pkl')
# joblib.dump(yield_scaler, 'yield_scaler.pkl')
# joblib.dump(quality_scaler, 'quality_scaler.pkl')

# # Also save the PyTorch wrapped versions
# torch.save(moisture_pt_model.state_dict(), 'moisture_pt_model.pt')
# torch.save(yield_pt_model.state_dict(), 'yield_pt_model.pt')
# torch.save(quality_pt_model.state_dict(), 'quality_pt_model.pt')

# print("All models successfully trained and saved!")

# # Function to load models for future use
# def load_models():
#     moisture_model = joblib.load('moisture_rf_model.pkl')
#     yield_model = joblib.load('yield_rf_model.pkl')
#     quality_model = joblib.load('quality_rf_model.pkl')
    
#     moisture_scaler = joblib.load('moisture_scaler.pkl')
#     yield_scaler = joblib.load('yield_scaler.pkl')
#     quality_scaler = joblib.load('quality_scaler.pkl')
    
#     # Create PyTorch wrappers
#     moisture_pt_model = RandomForestWrapper(moisture_model)
#     yield_pt_model = RandomForestWrapper(yield_model)
#     quality_pt_model = RandomForestWrapper(quality_model)
    
#     return {
#         'moisture': {
#             'model': moisture_model,
#             'pt_model': moisture_pt_model,
#             'scaler': moisture_scaler
#         },
#         'yield': {
#             'model': yield_model,
#             'pt_model': yield_pt_model,
#             'scaler': yield_scaler
#         },
#         'quality': {
#             'model': quality_model,
#             'pt_model': quality_pt_model,
#             'scaler': quality_scaler
#         }
#     }

Using device: cuda
Loading coffee dataset...
Dataset shape: (2394, 31)
Checking for NaN values: 0

===== Training Moisture Model =====
Moisture Model R²: 0.5789 (trained in 0.26 seconds)

===== Training Yield Model =====
Yield Model R²: 0.8809 (trained in 0.30 seconds)

===== Training Quality Model =====
Quality Model Mean R²: 0.7131 (trained in 3.03 seconds)
Individual R² scores:
  Aroma: 0.7314
  Flavor: 0.7190
  Aftertaste: 0.7292
  Acidity: 0.7537
  Body: 0.7752
  Balance: 0.7554
  Uniformity: 0.7369
  Clean Cup: 0.4539
  Sweetness: 0.7594
  Cupper Points: 0.7166

===== Optimizing Moisture Model =====
Optimizing model for Moisture...
Found better model: R² = 0.5762, Params: {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2}
Found better model: R² = 0.5789, Params: {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 2}
Found better model: R² = 0.5794, Params: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 2}
Optimized Moisture Model R²: 0.5794
B

# xg

In [23]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import xgboost as xgb
import matplotlib.pyplot as plt
import time
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Custom Dataset class for PyTorch integration
class CoffeeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# PyTorch wrapper for XGBoost model
class XGBoostWrapper(nn.Module):
    def __init__(self, xgb_model):
        super(XGBoostWrapper, self).__init__()
        self.xgb_model = xgb_model
        
    def forward(self, x):
        # Convert tensor to numpy for XGBoost
        x_np = x.detach().cpu().numpy()
        
        # Use DMatrix for faster prediction
        dmatrix = xgb.DMatrix(x_np)
        
        # Get predictions from XGBoost model
        pred_np = self.xgb_model.predict(dmatrix)
        
        # Convert back to tensor
        pred_tensor = torch.tensor(pred_np, dtype=torch.float32).to(x.device)
        
        return pred_tensor

# train an XGBoost model
def train_xgb_model(X_train, y_train, X_test, y_test, multi_output=False, 
                   params=None, num_boost_round=100, early_stopping_rounds=20):
    start_time = time.time()
    
    # Default params
    if params is None:
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'eta': 0.1,
            'max_depth': 6,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'seed': 42
        }
    
    # For multi-output regression, train multiple models
    if multi_output:
        models = []
        r2_scores = []
        
        for i in range(y_train.shape[1]):
            # Train a model for each output dimension
            dtrain = xgb.DMatrix(X_train, y_train[:, i])
            dtest = xgb.DMatrix(X_test, y_test[:, i])
            
            # Train with early stopping
            evals = [(dtrain, 'train'), (dtest, 'eval')]
            model = xgb.train(params, dtrain, num_boost_round=num_boost_round,
                             evals=evals, early_stopping_rounds=early_stopping_rounds,
                             verbose_eval=False)
            
            # Save model
            models.append(model)
            
            # Calculate R^2 score
            y_pred = model.predict(dtest)
            r2 = r2_score(y_test[:, i], y_pred)
            r2_scores.append(r2)
        
        # Calculate mean R^2
        mean_r2 = np.mean(r2_scores)
        
        # Create a "meta-model" that contains all individual models
        model = models
    else:
        # For single output regression, train one model
        if len(y_train.shape) > 1 and y_train.shape[1] == 1:
            y_train = y_train.ravel()
            y_test = y_test.ravel()
        
        # Create DMatrix objects
        dtrain = xgb.DMatrix(X_train, y_train)
        dtest = xgb.DMatrix(X_test, y_test)
        
        # Train with early stopping
        evals = [(dtrain, 'train'), (dtest, 'eval')]
        model = xgb.train(params, dtrain, num_boost_round=num_boost_round,
                         evals=evals, early_stopping_rounds=early_stopping_rounds,
                         verbose_eval=False)
        
        # Calculate R^2 score
        y_pred = model.predict(dtest)
        r2_scores = [r2_score(y_test, y_pred)]
        mean_r2 = r2_scores[0]
    
    train_time = time.time() - start_time
    
    return model, r2_scores, mean_r2, train_time

# Function to optimize hyperparameters and train best model
def optimize_xgb_model(X_train, y_train, X_test, y_test, multi_output=False, target_name=""):
    print(f"Optimizing XGBoost model for {target_name}...")
    
    best_r2 = -float('inf')
    best_params = None
    best_model = None
    best_r2_scores = None
    
    # Grid of parameters to try
    param_grid = [
        # Fast learning rate, different depths
        {'eta': 0.1, 'max_depth': 3, 'subsample': 0.8, 'colsample_bytree': 0.8},
        {'eta': 0.1, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8},
        {'eta': 0.1, 'max_depth': 9, 'subsample': 0.8, 'colsample_bytree': 0.8},
        
        # Medium learning rate, different regularization
        {'eta': 0.05, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.1},
        {'eta': 0.05, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_lambda': 1.0},
        
        # Slower learning rate, different sampling strategies
        {'eta': 0.01, 'max_depth': 6, 'subsample': 0.7, 'colsample_bytree': 0.9},
        {'eta': 0.01, 'max_depth': 6, 'subsample': 0.9, 'colsample_bytree': 0.7},
        
        # Deeper trees with regularization
        {'eta': 0.05, 'max_depth': 12, 'subsample': 0.8, 'colsample_bytree': 0.8, 
         'reg_alpha': 0.5, 'reg_lambda': 0.5},
    ]
    
    # Try all parameter combinations
    for params in param_grid:
        # Add common parameters
        full_params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'seed': 42,
            **params
        }
        
        # Train and evaluate model
        model, r2_scores, mean_r2, _ = train_xgb_model(
            X_train, y_train, X_test, y_test, 
            multi_output=multi_output, 
            params=full_params,
            num_boost_round=300,
            early_stopping_rounds=30
        )
        
        # Save if better than current best
        if mean_r2 > best_r2:
            best_r2 = mean_r2
            best_params = full_params
            best_model = model
            best_r2_scores = r2_scores
            
            # Print progress
            print(f"Found better model: R² = {mean_r2:.4f}, Params: {params}")
            
            # If we've achieved a really good score, we can stop early
            if best_r2 > 0.85:
                print("Found excellent model, stopping early.")
                break
    
    return best_model, best_r2_scores, best_r2, best_params

# Function to plot feature importance for XGBoost
def plot_feature_importance(model, feature_names, title, multi_output=False):
    plt.figure(figsize=(12, 8))
    
    # Get feature importances
    if multi_output:
        # For multi-output, average feature importance across models
        importances = np.zeros(len(feature_names))
        for m in model:
            # Get importance from the model
            importance_map = m.get_score(importance_type='gain')
            for f, imp in importance_map.items():
                # XGBoost uses f0, f1, etc. as feature names
                idx = int(f[1:])  # Extract the index from feature name
                importances[idx] += imp
        # Normalize
        importances = importances / len(model) if len(model) > 0 else importances
    else:
        # Get importance from the model
        importance_map = model.get_score(importance_type='gain')
        importances = np.zeros(len(feature_names))
        for f, imp in importance_map.items():
            # XGBoost uses f0, f1, etc. as feature names
            idx = int(f[1:])
            importances[idx] = imp
    
    # Sort by importance
    if np.sum(importances) > 0:
        importances = importances / np.sum(importances)
    
    # Sort indices by importance
    indices = np.argsort(importances)[::-1]
    
    # Plot top 15 features or all if fewer
    n_features = min(15, len(feature_names))
    plt.title(f'Feature Importance: {title}')
    plt.barh(range(n_features), importances[indices[:n_features]])
    plt.yticks(range(n_features), [feature_names[i] for i in indices[:n_features]])
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_").lower()}_feature_importance.png')
    plt.close()
    
    # Return as dictionary
    importance_dict = {feature_names[i]: importance for i, importance in enumerate(importances)}
    sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_importance)

# Function to make predictions with XGBoost
def predict_xgb(model, X, multi_output=False):
    # Create DMatrix for faster prediction
    dmatrix = xgb.DMatrix(X)
    
    if multi_output:
        # For multi-output, predict with each model
        predictions = np.column_stack([m.predict(dmatrix) for m in model])
        return predictions
    else:
        # For single output, predict with the model
        return model.predict(dmatrix)

# Main execution
print("Loading coffee dataset...")
df = pd.read_csv("final_coffee.csv")
print(f"Dataset shape: {df.shape}")

# Check for missing values
print(f"Checking for NaN values: {df.isna().sum().sum()}")

# Handle missing values if any
if df.isna().sum().sum() > 0:
    print("Filling missing values...")
    df = df.fillna(df.mean())

# Define features and targets
env_features = ["Annual precipitation", "Temperature Change on Land", "Altitude"]
processing_features = [col for col in df.columns if col.startswith("Processing Method_")]
origin_features = [col for col in df.columns if col.startswith("Country of Origin_")]
extra_cols = ["Category One Defects", "Quakers", "Category Two Defects"]

quality_input_cols = env_features + extra_cols + processing_features + origin_features
moisture_input_cols = env_features + processing_features + origin_features
yield_input_cols = env_features + processing_features + origin_features

quality_target_cols = ["Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance", 
                      "Uniformity", "Clean Cup", "Sweetness", "Cupper Points"]
moisture_target_col = "Moisture"
yield_target_col = "Total Yield"

# Prepare data

# Moisture Model
print("\n===== Training Moisture Model =====")
X_moisture = df[moisture_input_cols].values
y_moisture = df[[moisture_target_col]].values

# Split data
X_moisture_train, X_moisture_test, y_moisture_train, y_moisture_test = train_test_split(
    X_moisture, y_moisture, test_size=0.2, random_state=42
)

# Scale features
moisture_scaler = StandardScaler()
X_moisture_train_scaled = moisture_scaler.fit_transform(X_moisture_train)
X_moisture_test_scaled = moisture_scaler.transform(X_moisture_test)

# Train basic model first
moisture_model, moisture_r2_scores, moisture_r2, moisture_time = train_xgb_model(
    X_moisture_train_scaled, y_moisture_train, X_moisture_test_scaled, y_moisture_test,
    multi_output=False
)

print(f"Moisture Model R²: {moisture_r2:.4f} (trained in {moisture_time:.2f} seconds)")

# Yield Model
print("\n===== Training Yield Model =====")
X_yield = df[yield_input_cols].values
y_yield = df[[yield_target_col]].values

# Split data
X_yield_train, X_yield_test, y_yield_train, y_yield_test = train_test_split(
    X_yield, y_yield, test_size=0.2, random_state=42
)

# Scale features
yield_scaler = StandardScaler()
X_yield_train_scaled = yield_scaler.fit_transform(X_yield_train)
X_yield_test_scaled = yield_scaler.transform(X_yield_test)

# Train basic model first
yield_model, yield_r2_scores, yield_r2, yield_time = train_xgb_model(
    X_yield_train_scaled, y_yield_train, X_yield_test_scaled, y_yield_test,
    multi_output=False
)

print(f"Yield Model R²: {yield_r2:.4f} (trained in {yield_time:.2f} seconds)")

# Quality Model
print("\n===== Training Quality Model =====")
X_quality = df[quality_input_cols].values
y_quality = df[quality_target_cols].values

# Split data
X_quality_train, X_quality_test, y_quality_train, y_quality_test = train_test_split(
    X_quality, y_quality, test_size=0.2, random_state=42
)

# Scale features
quality_scaler = StandardScaler()
X_quality_train_scaled = quality_scaler.fit_transform(X_quality_train)
X_quality_test_scaled = quality_scaler.transform(X_quality_test)

# Train basic model first
quality_model, quality_r2_scores, quality_r2, quality_time = train_xgb_model(
    X_quality_train_scaled, y_quality_train, X_quality_test_scaled, y_quality_test,
    multi_output=True
)

print(f"Quality Model Mean R²: {quality_r2:.4f} (trained in {quality_time:.2f} seconds)")
print("Individual R^2 scores:")
for i, col in enumerate(quality_target_cols):
    print(f"  {col}: {quality_r2_scores[i]:.4f}")

# Define the R² threshold for model optimization
r2_threshold = 0.7

# Optimize models if needed
if moisture_r2 < r2_threshold:
    print("\n===== Optimizing Moisture Model =====")
    moisture_model, moisture_r2_scores, moisture_r2, moisture_params = optimize_xgb_model(
        X_moisture_train_scaled, y_moisture_train, X_moisture_test_scaled, y_moisture_test,
        multi_output=False, target_name="Moisture"
    )
    print(f"Optimized Moisture Model R²: {moisture_r2:.4f}")
    print(f"Best parameters: {moisture_params}")

if yield_r2 < r2_threshold:
    print("\n===== Optimizing Yield Model =====")
    yield_model, yield_r2_scores, yield_r2, yield_params = optimize_xgb_model(
        X_yield_train_scaled, y_yield_train, X_yield_test_scaled, y_yield_test,
        multi_output=False, target_name="Yield"
    )
    print(f"Optimized Yield Model R²: {yield_r2:.4f}")
    print(f"Best parameters: {yield_params}")

if quality_r2 < r2_threshold:
    print("\n===== Optimizing Quality Model =====")
    quality_model, quality_r2_scores, quality_r2, quality_params = optimize_xgb_model(
        X_quality_train_scaled, y_quality_train, X_quality_test_scaled, y_quality_test,
        multi_output=True, target_name="Quality"
    )
    print(f"Optimized Quality Model Mean R²: {quality_r2:.4f}")
    print(f"Best parameters: {quality_params}")
    print("Individual R² scores:")
    for i, col in enumerate(quality_target_cols):
        print(f"  {col}: {quality_r2_scores[i]:.4f}")

# Feature importance analysis
print("\n===== Feature Importance Analysis =====")

# Moisture model feature importance
moisture_importance = plot_feature_importance(
    moisture_model, moisture_input_cols, "Moisture Model"
)
print("\nTop 5 features for Moisture Model:")
for i, (feature, importance) in enumerate(list(moisture_importance.items())[:5]):
    print(f"  {i+1}. {feature}: {importance:.4f}")

# Yield model feature importance
yield_importance = plot_feature_importance(
    yield_model, yield_input_cols, "Yield Model (XGBoost)"
)
print("\nTop 5 features for Yield Model:")
for i, (feature, importance) in enumerate(list(yield_importance.items())[:5]):
    print(f"  {i+1}. {feature}: {importance:.4f}")

multi_output = isinstance(quality_model, list)

if multi_output:
    quality_importance = plot_feature_importance(
        quality_model, quality_input_cols, "Quality Model", multi_output=True
    )
else:
    quality_importance = plot_feature_importance(
        quality_model[0] if isinstance(quality_model, list) else quality_model, 
        quality_input_cols, "Quality Model"
    )
    
print("\nTop 5 features for Quality Model:")
for i, (feature, importance) in enumerate(list(quality_importance.items())[:5]):
    print(f"  {i+1}. {feature}: {importance:.4f}")

# Wrap the XGBoost models with PyTorch for compatibility
class XGBoostMultiOutputWrapper(nn.Module):
    def __init__(self, models):
        super(XGBoostMultiOutputWrapper, self).__init__()
        self.models = models
        
    def forward(self, x):
        # Convert tensor to numpy for XGBoost
        x_np = x.detach().cpu().numpy()
        dmatrix = xgb.DMatrix(x_np)
        
        # Get predictions from each XGBoost model
        preds = np.column_stack([m.predict(dmatrix) for m in self.models])
        
        # Convert back to tensor
        pred_tensor = torch.tensor(preds, dtype=torch.float32).to(x.device)
        
        return pred_tensor

# Create PyTorch wrappers
if isinstance(moisture_model, list):
    moisture_pt_model = XGBoostMultiOutputWrapper(moisture_model)
else:
    moisture_pt_model = XGBoostWrapper(moisture_model)
    
if isinstance(yield_model, list):
    yield_pt_model = XGBoostMultiOutputWrapper(yield_model)
else:
    yield_pt_model = XGBoostWrapper(yield_model)
    
if isinstance(quality_model, list):
    quality_pt_model = XGBoostMultiOutputWrapper(quality_model)
else:
    quality_pt_model = XGBoostWrapper(quality_model)

# Sample predictions
print("\n===== Sample Predictions =====")
sample_data = df.sample(5, random_state=42)

# Prepare data for predictions
sample_moisture_X = moisture_scaler.transform(sample_data[moisture_input_cols].values)
sample_yield_X = yield_scaler.transform(sample_data[yield_input_cols].values)
sample_quality_X = quality_scaler.transform(sample_data[quality_input_cols].values)

# Make predictions with XGBoost models directly
moisture_preds = predict_xgb(moisture_model, sample_moisture_X, multi_output=isinstance(moisture_model, list))
yield_preds = predict_xgb(yield_model, sample_yield_X, multi_output=isinstance(yield_model, list))
quality_preds = predict_xgb(quality_model, sample_quality_X, multi_output=isinstance(quality_model, list))

# Display predictions
print("\nSample Predictions:")
for i in range(len(sample_data)):
    print(f"\nSample {i+1}:")
    print(f"  Altitude: {sample_data['Altitude'].iloc[i]:.2f}")
    
    # Get actual and predicted values
    moisture_actual = sample_data[moisture_target_col].iloc[i]
    moisture_pred = moisture_preds[i] if moisture_preds.ndim > 1 else moisture_preds[i]
    
    yield_actual = sample_data[yield_target_col].iloc[i]
    yield_pred = yield_preds[i] if yield_preds.ndim > 1 else yield_preds[i]
    
    print(f"  Moisture - Actual: {moisture_actual:.2f}, Predicted: {moisture_pred:.2f}")
    print(f"  Yield - Actual: {yield_actual:.2f}, Predicted: {yield_pred:.2f}")
    
    print("  Quality Metrics:")
    for j, col in enumerate(quality_target_cols):
        actual = sample_data[col].iloc[i]
        predicted = quality_preds[i, j] if quality_preds.ndim > 1 else quality_preds[j][i]
        print(f"    {col}: Actual: {actual:.2f}, Predicted: {predicted:.2f}")

# Define a PyTorch modeling class that uses the XGBoost models
class CoffeeQualityPredictor(nn.Module):
    def __init__(self, moisture_model, yield_model, quality_model,
                moisture_scaler, yield_scaler, quality_scaler):
        super(CoffeeQualityPredictor, self).__init__()
        
        # Store scalers
        self.moisture_scaler = moisture_scaler
        self.yield_scaler = yield_scaler
        self.quality_scaler = quality_scaler
        
        # Create PyTorch wrappers for XGBoost models
        if isinstance(moisture_model, list):
            self.moisture_model = XGBoostMultiOutputWrapper(moisture_model)
        else:
            self.moisture_model = XGBoostWrapper(moisture_model)
            
        if isinstance(yield_model, list):
            self.yield_model = XGBoostMultiOutputWrapper(yield_model)
        else:
            self.yield_model = XGBoostWrapper(yield_model)
            
        if isinstance(quality_model, list):
            self.quality_model = XGBoostMultiOutputWrapper(quality_model)
        else:
            self.quality_model = XGBoostMultiOutputWrapper([quality_model]) 
            
        # Define feature columns
        self.moisture_cols = moisture_input_cols
        self.yield_cols = yield_input_cols
        self.quality_cols = quality_input_cols
        self.quality_target_cols = quality_target_cols
        
    def preprocess_features(self, features_dict):
        """Convert dictionary of features to scaled tensors for each model"""
        
        # Extract features for each model
        moisture_features = np.array([[features_dict[col] for col in self.moisture_cols]])
        yield_features = np.array([[features_dict[col] for col in self.yield_cols]])
        quality_features = np.array([[features_dict[col] for col in self.quality_cols]])
        
        # Scale features
        moisture_scaled = self.moisture_scaler.transform(moisture_features)
        yield_scaled = self.yield_scaler.transform(yield_features)
        quality_scaled = self.quality_scaler.transform(quality_features)
        
        # Convert to tensors
        moisture_tensor = torch.tensor(moisture_scaled, dtype=torch.float32)
        yield_tensor = torch.tensor(yield_scaled, dtype=torch.float32)
        quality_tensor = torch.tensor(quality_scaled, dtype=torch.float32)
        
        return moisture_tensor, yield_tensor, quality_tensor
    
    def forward(self, features_dict):
        """Predict coffee quality from feature dictionary"""
        
        # Preprocess features
        moisture_tensor, yield_tensor, quality_tensor = self.preprocess_features(features_dict)
        
        # Get predictions
        moisture_pred = self.moisture_model(moisture_tensor)
        yield_pred = self.yield_model(yield_tensor)
        quality_pred = self.quality_model(quality_tensor)
        
        # Create results dictionary
        results = {
            'moisture': moisture_pred.item(),
            'yield': yield_pred.item(),
            'quality': {}
        }
        
        # Add quality predictions
        for i, col in enumerate(self.quality_target_cols):
            results['quality'][col] = quality_pred[0, i].item() if quality_pred.dim() > 1 else quality_pred[0].item()
            
        return results

# Create and save the unified model
unified_model = CoffeeQualityPredictor(
    moisture_model, yield_model, quality_model,
    moisture_scaler, yield_scaler, quality_scaler
)

# Save the models and scalers
print("\n===== Saving Models =====")
joblib.dump(moisture_model, 'xgb_moisture_model.pkl')
joblib.dump(yield_model, 'xgb_yield_model.pkl')
joblib.dump(quality_model, 'xgb_quality_model.pkl')

joblib.dump(moisture_scaler, 'xgb_moisture_scaler.pkl')
joblib.dump(yield_scaler, 'xgb_yield_scaler.pkl')
joblib.dump(quality_scaler, 'xgb_quality_scaler.pkl')

# Save the PyTorch wrapper model
torch.save(unified_model.state_dict(), 'coffee_quality_xgboost_pytorch.pt')

print("All models successfully trained and saved!")


# Create a sample feature dictionary
sample_features = {
    'Altitude': 1500,
    'Annual precipitation': 1200,
    'Temperature Change on Land': 1.2,
    'Category One Defects': 0,
    'Quakers': 0,
    'Category Two Defects': 0
}

# Add processing method features (all 0 except for "Washed / Wet")
for method in processing_features:
    sample_features[method] = 1.0 if method == "Processing Method_Washed / Wet" else 0.0

# Add origin features (all 0 except for a chosen country)
for origin in origin_features:
    sample_features[origin] = 1.0 if "Ethiopia" in origin else 0.0

# Make a prediction with the PyTorch model
unified_model.eval()
with torch.no_grad():
    try:
        prediction = unified_model(sample_features)
        print("PyTorch Model Prediction:")
        print(f"  Predicted Moisture: {prediction['moisture']:.2f}")
        print(f"  Predicted Yield: {prediction['yield']:.2f}")
        print("  Predicted Quality Metrics:")
        for metric, value in prediction['quality'].items():
            print(f"    {metric}: {value:.2f}")
    except Exception as e:
        print(f"Error in PyTorch prediction: {e}")
        print("This is likely due to feature mismatch. Please adjust the example to match your dataset.")

print("\ncompleted successfully!")

# Function to load models for future use
def load_coffee_model():
    """Load the saved coffee quality prediction model"""
    # Load XGBoost models
    moisture_model = joblib.load('xgb_moisture_model.pkl')
    yield_model = joblib.load('xgb_yield_model.pkl')
    quality_model = joblib.load('xgb_quality_model.pkl')
    
    # Load scalers
    moisture_scaler = joblib.load('xgb_moisture_scaler.pkl')
    yield_scaler = joblib.load('xgb_yield_scaler.pkl')
    quality_scaler = joblib.load('xgb_quality_scaler.pkl')
    
    # Create unified model
    unified_model = CoffeeQualityPredictor(
        moisture_model, yield_model, quality_model,
        moisture_scaler, yield_scaler, quality_scaler
    )
    
    return unified_model

Using device: cuda
Loading coffee dataset...
Dataset shape: (2394, 31)
Checking for NaN values: 0

===== Training Moisture Model =====
Moisture Model R²: 0.5011 (trained in 0.09 seconds)

===== Training Yield Model =====
Yield Model R²: 0.9283 (trained in 0.08 seconds)

===== Training Quality Model =====
Quality Model Mean R²: 0.7542 (trained in 0.83 seconds)
Individual R^2 scores:
  Aroma: 0.7398
  Flavor: 0.7591
  Aftertaste: 0.7235
  Acidity: 0.7808
  Body: 0.7942
  Balance: 0.7507
  Uniformity: 0.7872
  Clean Cup: 0.6211
  Sweetness: 0.8781
  Cupper Points: 0.7077

===== Optimizing Moisture Model =====
Optimizing XGBoost model for Moisture...
Found better model: R² = 0.3816, Params: {'eta': 0.1, 'max_depth': 3, 'subsample': 0.8, 'colsample_bytree': 0.8}
Found better model: R² = 0.5695, Params: {'eta': 0.1, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8}
Found better model: R² = 0.5919, Params: {'eta': 0.1, 'max_depth': 9, 'subsample': 0.8, 'colsample_bytree': 0.8}
Optimi

In [ ]:
tabnet